In [1]:
import numpy as np
import os
import pickle
import re
import random

import tensorflow as tf
from tensorflow.contrib import rnn
import tensorflow.contrib.slim as slim
from tqdm import tqdm

# model.py

In [2]:
tf.reset_default_graph()
graph = tf.Graph()

In [3]:
with open('asin_dict.pkl', 'rb') as f:
    asin_dict = pickle.load(f)

In [4]:
time_window = 10
max_len_review = 3000
char_mtx_row = 1000
_len_alphabet = 70
_meta_dim = 85
item_dim = len(asin_dict[list(asin_dict.keys())[0]])
batch_size = 5

In [5]:
char_mtx = tf.placeholder(dtype=tf.float32, shape=[None, time_window, max_len_review, _len_alphabet])
img_mtx = tf.placeholder(dtype=tf.float32, shape=[None, time_window, 32,32,3]) # input image shape: 32x32
meta = tf.placeholder(dtype=tf.float32, shape=[None, time_window, _meta_dim])
y = tf.placeholder(dtype=tf.float32, shape=[None, item_dim])

In [6]:
def review_CNN(char_mtx, filter_sizes = [2, 3, 7, 11], filter_nums = [100, 100, 100, 100], stride=[1,1,1,1],
               mlp_units=[200,50], reuse=False, is_training=True):
    char_mtx = tf.reshape(char_mtx, [-1, max_len_review, _len_alphabet])
    ## CNN for review aspect, sentiment extraction
    with tf.variable_scope('review-charCNN', reuse=reuse):
        input_ = tf.expand_dims(char_mtx, axis=3)
        with tf.name_scope('conv-filter-1'):
            filter_shape1 = [filter_sizes[0], _len_alphabet, 1, filter_nums[0]]
            W1 = tf.Variable(tf.truncated_normal(shape=filter_shape1), name='filter-1')
            b1 = tf.Variable(tf.random_uniform(shape=[filter_nums[0]]), name='bias-1')
            conv1 = tf.nn.conv2d(input_, W1, strides=stride, padding='VALID', name='conv-1')
            out1 = tf.nn.relu(tf.nn.bias_add(conv1, b1))
            max1 = tf.reduce_max(out1, axis=1) # batch_size, 1, 100
        with tf.name_scope('conv-filter-2'):
            filter_shape2= [filter_sizes[1], _len_alphabet, 1, filter_nums[1]]
            W2 = tf.Variable(tf.truncated_normal(shape=filter_shape2), name='filter-2')
            b2 = tf.Variable(tf.random_uniform(shape=[filter_nums[1]]), name='bias-2')
            conv2 = tf.nn.conv2d(input_, W2, strides=stride, padding='VALID', name='conv-2')
            out2 = tf.nn.relu(tf.nn.bias_add(conv2, b2))
            max2 = tf.reduce_max(out2, axis=1) # batch_size, 1, 100
        with tf.name_scope('conv-filter-3'):
            filter_shape3 = [filter_sizes[2], _len_alphabet, 1, filter_nums[2]]
            W3 = tf.Variable(tf.truncated_normal(shape=filter_shape3), name='filter-3')
            b3 = tf.Variable(tf.random_uniform(shape=[filter_nums[2]]), name='bias-3')
            conv3 = tf.nn.conv2d(input_, W3, strides=stride, padding='VALID', name='conv-3')
            out3 = tf.nn.relu(tf.nn.bias_add(conv3, b3))
            max3 = tf.reduce_max(out3, axis=1) # batch_size, 1, 100
        with tf.name_scope('conv-filter-4'):
            filter_shape4= [filter_sizes[3], _len_alphabet, 1, filter_nums[3]]
            W4 = tf.Variable(tf.truncated_normal(shape=filter_shape4), name='filter-4')
            b4 = tf.Variable(tf.random_uniform(shape=[filter_nums[3]]), name='bias-4')
            conv4 = tf.nn.conv2d(input_, W4, strides=stride, padding='VALID', name='conv-4')
            out4 = tf.nn.relu(tf.nn.bias_add(conv4, b4))
            max4 = tf.reduce_max(out4, axis=1) # batch_size, 1, 100
        ## concat
        max_concat = tf.squeeze(tf.concat([max1, max2, max3, max4], axis=2), axis=1) # batch_size, 400
    ## MLP for feature reduction
    with tf.variable_scope('review-MLP', reuse=reuse):
        fc1 = slim.fully_connected(max_concat, mlp_units[0])
        fc2 = slim.fully_connected(fc1, mlp_units[1])
    result = tf.reshape(fc2, shape=[-1, mlp_units[1]])
    return result

In [7]:
def image_CNN(img_mtx, mlp_units=[200,50], reuse=False, is_training=True):
    img_mtx = tf.reshape(img_mtx, [-1, 32, 32, 3])
    with tf.variable_scope('image-CNN', reuse=reuse):
        with slim.arg_scope([slim.conv2d], padding='SAME', activation_fn=None, 
                    stride = 2, weights_initializer=tf.contrib.layers.xavier_initializer()):
            with slim.arg_scope([slim.batch_norm], decay=0.95, center=True, scale=True,
                            updates_collections = None, activation_fn=tf.nn.relu,
                            is_training =is_training):
                conv1 = slim.conv2d(img_mtx, 6, [3,3], scope='conv-1')
                bn1 = slim.batch_norm(conv1, scope='bn-1')
                conv2 = slim.conv2d(bn1, 12, [3,3], scope='conv-2')
                bn2 = slim.batch_norm(conv2, scope='bn-2')
        with tf.variable_scope('MLP', reuse=reuse):
            fc1 = slim.fully_connected(slim.flatten(bn2), mlp_units[0])
            fc2 = slim.fully_connected(fc1, mlp_units[1])
    return fc2

In [8]:
def userLSTM(lstm_input, hidden=128, reuse=False):
    cell = rnn.BasicLSTMCell(hidden, reuse=reuse)
    input_ = tf.unstack(lstm_input, axis=1)    
    outputs, _ = rnn.static_rnn(cell, input_, dtype=tf.float32, scope='juungLSTM')
    return outputs[-1]

In [9]:
def fc_pred(lstm_result, reuse=False):
    fc1 = slim.fully_connected(lstm_result, 1024, scope='fc-1')
    fc2 = slim.fully_connected(fc1, item_dim, scope='fc-2')
    return fc2

In [10]:
def juung(char_mtx, img_mtx, meta, hidden=128, reuse=False):
    review_result = review_CNN(char_mtx, reuse=False)
    img_result = image_CNN(img_mtx)
    meta = tf.reshape(meta, [-1, 85])
    concat_result = tf.concat([review_result, img_result, meta], axis=1)
    lstm_input = tf.reshape(concat_result, shape=[-1, time_window, 185]) # 185 = 50+50+meta_dim
    lstm_result = userLSTM(lstm_input, hidden=hidden)
    pred = fc_pred(lstm_result)
    return pred

In [11]:
pred = juung(char_mtx, img_mtx, meta)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
y_idx = tf.cast(tf.argmax(y, axis=1), dtype=tf.int32)
top_k = tf.reduce_mean(tf.cast(tf.nn.in_top_k(pred, y_idx, k=10, name='top-k'), dtype=tf.float32))

Tensor("Reshape_1:0", shape=(?, 50), dtype=float32)
Tensor("image-CNN/MLP/fully_connected_1/Relu:0", shape=(?, 50), dtype=float32)
Tensor("Placeholder_2:0", shape=(?, 10, 85), dtype=float32)
Tensor("concat:0", shape=(?, 185), dtype=float32)


# Solver.py

https://github.com/yunjey/domain-transfer-network/blob/master/solver.py  

class solver():
- init
- load_data
- batch_iter
- train
- eval(test)

In [12]:
def __init__():
    data_path = 'data'
    batch_size = 5
    train_iter = 10
    log_dir = 'logs/'
    model_save = 'model/'

In [13]:
def review_to_mtx(review, row_size=max_len_review):
    """
    review to character-level matrix
    """
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}\n"
    review = review[0].lower()
    if len(review) > row_size:
        review = review[:row_size]
    char_mtx = np.zeros([row_size, len(alphabet)], dtype=np.float32)
    for i, char in enumerate(review):
        if char in alphabet:
            char_mtx[i, alphabet.index(char)] = 1
    return char_mtx

In [14]:
def read_data(mode):
    '''
    mode is 'train', 'val' or 'test'
    '''
    file_list = [x for x in os.listdir('data') if mode in x]
    file_name = random.sample(file_list, k=1)
    print("Opening... {}".format(file_name[0]))
    with open('data/'+file_name[0], 'rb') as f:
        data = pickle.load(f)
    print("Done!")
    return data

In [15]:
def batch_iter(data, asin_dict, batch_size, num_epochs = 2, shuffle = True):
    # data[0]: 1000 length list, each list is array of shape (10, 32, 32, 3)
    img = np.array(data[0]) # shape: (1000, 10, 32, 32, 3)
    review = np.array(data[1]) # shape : (1000, 10, string)
    meta = np.array(data[2]) # shape : (1000, 10, 85)
    asin = np.array(data[3]) # shape : (1000, 1)
    data_size = len(data[0])
    num_batches_per_epoch = int(data_size / batch_size) + 1
    
    for epoch in range(num_epochs):
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            img_ , review_, meta_, asin_ = img[shuffle_indices], review[shuffle_indices], meta[shuffle_indices], asin[shuffle_indices]
        else:
            img_ , review_, meta_, asin_ = img, review, meta, asin
        
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = (batch_num + 1) * batch_size
            if end_index < data_size:
                img_batch = img_[start_index:end_index] 
                meta_batch = meta_[start_index:end_index]
                review_batch = np.reshape(review_[start_index:end_index], [-1, 1])
                review_batch_transform = []
                for index in range(review_batch.shape[0]):
                    review_batch_transform.append(review_to_mtx(review_batch[index]))
                review_batch_transform = np.reshape(np.array(review_batch_transform), [-1,10,max_len_review,_len_alphabet])
                
                asin_batch = asin_[start_index:end_index]
                asin_batch_encode = []
                for index in range(asin_batch.shape[0]):
                    asin_batch_encode.append(asin_dict[asin_batch[index][0]])
                asin_batch_encode = np.array(asin_batch_encode)
                yield list(zip(img_batch, review_batch_transform, meta_batch, asin_batch_encode))            

In [16]:
train_data = read_data('train')

Opening... train_0.pkl
Done!


In [18]:
learning_rate = 2e-4

In [20]:
config = tf.ConfigProto(log_device_placement = True)
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
with sess.as_default():
    global_step = tf.Variable(0, name='global_step', trainable=False)
    opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
    sess.run(tf.global_variables_initializer())
    batch_train = batch_iter(train_data, asin_dict, batch_size=batch_size, num_epochs=2)
    for train in batch_train:
        img_batch, review_batch, meta_batch, asin_batch = zip(*train)
        feed_dict = {char_mtx : review_batch, img_mtx : img_batch, meta: meta_batch, y : asin_batch}
        current_step = sess.run(global_step, feed_dict = feed_dict)
        sess.run(opt, feed_dict=feed_dict)
        if current_step % 10 == 0:
            print("step: {}".format(current_step))
            print("====validation start====")
            batch_val = batch_iter(train_data, asin_dict, batch_size, 1)
            top_ks = []
            for val in batch_val:
                img_val, review_val, meta_val, asin_val = zip(*train)
                feed_dict = {char_mtx : review_val, img_mtx : img_val, meta: meta_val, y : asin_val}
                top_k_ = sess.run(top_k, feed_dict = feed_dict)
                top_ks.append(top_k_)
            print("Mean top_k = " + str(sum(top_ks)/len(top_ks)))
            print("===========training============")

step: 0
====validation start====
Mean top_k = 0.0
===========training============
Training finished.
step: 10
====validation start====
Mean top_k = 0.0
===========training============
Training finished.


KeyboardInterrupt: 